<a href="https://colab.research.google.com/github/Lambda-School-Labs/Labs26-Citrics-DS-TeamC/blob/mtoce/Weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Driftly Weather Notebook

## Import and Install Required Packages

In [1]:
# PACKAGES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import urllib.request
import urllib
import urllib.parse
import os
import requests
from datetime import datetime
# import prettytable
import seaborn as sns
# import plotly.express as px

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
pip install wwo-hist

In [3]:
from wwo_hist import retrieve_hist_data

In [4]:
capitals_list = ['Albany','Annapolis','Atlanta','Augusta','Austin','Baton_Rouge','Bismarck','Boise','Boston','Carson_City','Charleston','Cheyenne','Columbia','Columbus','Concord','Denver','Des_Moines','Dover','Frankfort','Harrisburg','Hartford','Helena','Honolulu','Indianapolis','Jackson','Jefferson_City','Juneau','Lansing','Lincoln','LittleRock','Madison','Montgomery','Montpelier','Nashville','Oklahoma_City','Olympia','Phoenix','Pierre','Providence','Raleigh','Richmond','Sacramento','Saint_Paul','Salem','Salt_Lake_City','Santa_Fe_USA','Springfield','Tallahassee','Topeka','Trenton']

states_list = ['Alabama','Alaska','Arizona','Arkansas','California','Colorado','Connecticut','Delaware','Florida','Georgia','Hawaii','Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana','Maine','Maryland','Massachusetts','Michigan','Minnesota','Mississippi','Missouri','Montana','Nebraska','Nevada','New_Hampshire','New_Jersey','New Mexico','New_York','North_Carolina','North_Dakota','Ohio','Oklahoma','Oregon','Pennsylvania','Rhode_Island','South_Carolina','South_Dakota','Tennessee','Texas','Utah','Vermont','Virginia','Washington','West_Virginia','Wisconsin','Wyoming']

In [5]:
# Got stopped because of query limit and need to query these cities still as of 9/08
capitals_list_retry = ['Santa_Fe_USA','Trenton']

## Data Aggregation using WWO API

In [6]:
# # USING THE API
# #########################################################
# frequency = 24  # time between each data row (hours) - max of 24
# start_date = '01-JAN-2015'
# end_date = '31-DEC-2018'
# free_trial_api_key = '831e61eaec474f5ca63202520200309'
# new_api_key = 'ddd3168a425b434f828201815200309'
# location_list = ['Santa_Fe_USA','Trenton']
# hist_weather_data = retrieve_hist_data(new_api_key,
#                                 location_list,
#                                 start_date,
#                                 end_date,
#                                 frequency,
#                                 location_label = False,
#                                 export_csv = True,
#                                 store_df = True)

## Clean the Data for Predictive Modeling

In [7]:
## FOR COLLAB USE

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import pandas as pd
import glob

local_path = r'C:\Users\Mike\LambdaSchool\labs\Labs26-Citrics-DS-TeamC\data\weather'
collab_path = r'/content/drive/My Drive/Lambda/weather'

all_files = glob.glob(collab_path + "/*.csv")

list_ = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    list_.append(df)

data = pd.concat(list_, axis=0, ignore_index=True)
data.tail()

,Unnamed: 0,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,sunset,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,location
91345,1822,2019-12-28,5,0,0.0,6.9,2,8,10:24 AM,07:57 PM,08:24 AM,05:29 PM,-1,-1,2,-1,17,51,83,0.0,1023,5,10,308,10,Albany_NY
91346,1823,2019-12-29,4,-1,0.0,8.7,2,15,11:02 AM,08:58 PM,08:25 AM,05:30 PM,-1,0,1,0,7,48,85,10.8,1022,4,9,189,4,Albany_NY
91347,1824,2019-12-30,5,1,0.0,3.4,1,22,11:34 AM,10:00 PM,08:25 AM,05:30 PM,2,1,3,1,13,97,93,40.1,1011,5,7,118,8,Albany_NY
91348,1825,2019-12-31,4,0,0.3,3.4,1,26,12:01 PM,11:00 PM,08:25 AM,05:31 PM,0,0,2,0,8,91,90,2.1,1003,4,6,255,6,Albany_NY
91349,1826,2020-01-01,2,-2,0.0,5.2,1,37,12:25 PM,12:00 AM,08:25 AM,05:32 PM,-5,-4,0,-4,15,40,70,0.0,1005,2,10,279,10,Albany_NY


In [9]:
data = data.drop('Unnamed: 0', axis=1)
data.head()

,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,sunset,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,location
0,2015-01-01,4,-4,0.0,8.7,2,72,03:15 PM,04:47 AM,08:21 AM,05:44 PM,-11,-5,-1,-5,27,8,48,0.0,1021,4,10,242,16,Trenton_NJ
1,2015-01-02,5,-1,0.0,8.7,2,80,04:01 PM,05:46 AM,08:21 AM,05:45 PM,-5,-2,1,-2,24,29,65,0.0,1025,5,10,272,13,Trenton_NJ
2,2015-01-03,7,-2,0.0,5.2,1,87,04:51 PM,06:41 AM,08:21 AM,05:46 PM,1,0,3,0,18,59,86,0.2,1031,7,7,84,10,Trenton_NJ
3,2015-01-04,14,6,0.0,3.5,3,94,05:44 PM,07:32 AM,08:22 AM,05:46 PM,9,9,10,9,26,85,93,5.8,1012,14,7,233,16,Trenton_NJ
4,2015-01-05,6,-4,0.0,7.0,2,100,06:39 PM,08:16 AM,08:22 AM,05:47 PM,-6,-4,1,-4,34,28,62,0.0,1022,6,10,286,23,Trenton_NJ


In [10]:
def clean(DataFrame):

    # copy frame
    X = DataFrame.copy()

    # drop unneeded cols
    X = X.drop(['winddirDegree','moonrise','moonset','moon_illumination','sunrise','sunset','sunHour','tempC'], axis=1)
    
    # create temp df to str split location into multiple cols then concat new DF
    loc_temp = X['location'].str.split('_', expand=True)
    loc_temp = loc_temp.rename(columns={0: "city", 1: "state_abbr"})
    X = pd.concat([X, loc_temp], axis=1)
    X = X.drop('location', axis=1)

    # rename cols from str.split
    X = X.rename(columns={0: "city", 1: "state_abbr"})

    # convert new columns to strings
    X['city'] = X['city'].astype('string')
    X['state_abbr'] = X['state_abbr'].astype('string')

    # replace dashes with spaces in city col
    X['city'] = X['city'].str.replace('-', ' ')

    # split date_time into categories for splitting df seasonally
    X['date_time'] = pd.to_datetime(X['date_time'], infer_datetime_format=True)
    X['year'] = X['date_time'].dt.year
    X['month'] = X['date_time'].dt.month
    X['day'] = X['date_time'].dt.day

    # date_temp = X['date_string'].str.split('-', expand=True)
    # date_temp = date_temp.rename(columns={0: "year", 1: "month", 2: "day"})
    # X = pd.concat([X, date_temp])


    # grab column headers
    cols = list(X)

    # rearrange column order to make most sense
    cols.insert(0, cols.pop(cols.index('city')))
    cols.insert(1, cols.pop(cols.index('state_abbr')))
    cols.insert(20, cols.pop(cols.index('date_time')))
    cols.insert(3, cols.pop(cols.index('FeelsLikeC')))
    cols.insert(4, cols.pop(cols.index('precipMM')))


    # save new reordered frame from rearranged cols list
    X = X.loc[:, cols]

    return X

In [11]:
data = clean(data)
data.head()

,city,state_abbr,maxtempC,FeelsLikeC,precipMM,mintempC,totalSnow_cm,uvIndex,DewPointC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,humidity,pressure,visibility,windspeedKmph,year,month,day,date_time
0,Trenton,NJ,4,-5,0.0,-4,0.0,2,-11,-1,-5,27,8,48,1021,10,16,2015,1,1,2015-01-01
1,Trenton,NJ,5,-2,0.0,-1,0.0,2,-5,1,-2,24,29,65,1025,10,13,2015,1,2,2015-01-02
2,Trenton,NJ,7,0,0.2,-2,0.0,1,1,3,0,18,59,86,1031,7,10,2015,1,3,2015-01-03
3,Trenton,NJ,14,9,5.8,6,0.0,3,9,10,9,26,85,93,1012,7,16,2015,1,4,2015-01-04
4,Trenton,NJ,6,-4,0.0,-4,0.0,2,-6,1,-4,34,28,62,1022,10,23,2015,1,5,2015-01-05


In [12]:
summer = data[(data['month'] >= 4) & (data['month'] <= 9)]
winter = data[(data['month'] < 4) | (data['month'] > 9)]
print(f"Summer shape: {summer.shape}  ;  Winter shape:{winter.shape}")

Summer shape: (45750, 21)  ;  Winter shape:(45600, 21)


In [13]:
summer.head()

,city,state_abbr,maxtempC,FeelsLikeC,precipMM,mintempC,totalSnow_cm,uvIndex,DewPointC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,humidity,pressure,visibility,windspeedKmph,year,month,day,date_time
90,Trenton,NJ,10,2,0.0,-1,0.0,2,-1,5,2,23,10,71,1019,10,16,2015,4,1,2015-04-01
91,Trenton,NJ,17,8,0.0,2,0.0,3,3,10,8,36,26,62,1021,10,23,2015,4,2,2015-04-02
92,Trenton,NJ,18,14,1.2,13,0.0,4,13,15,14,37,91,89,1010,9,23,2015,4,3,2015-04-03
93,Trenton,NJ,16,8,2.6,5,0.0,4,5,11,8,42,31,68,1012,10,29,2015,4,4,2015-04-04
94,Trenton,NJ,16,7,0.0,3,0.0,3,1,10,7,39,10,56,1023,10,24,2015,4,5,2015-04-05


In [14]:
psc_data = pd.read_csv('Postal_States_Capitals_CSV.csv')
# psc_data

postals = list(psc_data['Postal'])
states = list(psc_data['State'])
capitals = list(psc_data['Capital'])

# print(len(postals), len(states), len(capitals))

capitals_postals_dict = dict(zip(capitals, postals))
capitals_postals_dict

{'Albany': 'NY',
 'Annapolis': 'MD',
 'Atlanta': 'GA',
 'Augusta': 'ME',
 'Austin': 'TX',
 'Baton Rouge': 'LA',
 'Bismarck': 'ND',
 'Boise': 'ID',
 'Boston': 'MA',
 'Carson City': 'NV',
 'Charleston': 'WV',
 'Cheyenne': 'WY',
 'Columbia': 'SC',
 'Columbus': 'OH',
 'Concord': 'NH',
 'Denver': 'CO',
 'Des Moines': 'IA',
 'Dover': 'DE',
 'Frankfort': 'KY',
 'Harrisburg': 'PA',
 'Hartford': 'CT',
 'Helena': 'MT',
 'Honolulu': 'HI',
 'Indianapolis': 'IN',
 'Jackson': 'MS',
 'Jefferson City': 'MO',
 'Juneau': 'AK',
 'Lansing': 'MI',
 'Lincoln': 'NE',
 'Little Rock': 'AR',
 'Madison': 'WI',
 'Montgomery': 'AL',
 'Montpelier': 'VT',
 'Nashville': 'TN',
 'Oklahoma City': 'OK',
 'Olympia': 'WA',
 'Phoenix': 'AZ',
 'Pierre': 'SD',
 'Providence': 'RI',
 'Raleigh': 'NC',
 'Richmond': 'VA',
 'Sacramento': 'CA',
 'Saint Paul': 'MN',
 'Salem': 'OR',
 'Salt Lake City': 'UT',
 'Santa Fe': 'NM',
 'Springfield': 'IL',
 'Tallahassee': 'FL',
 'Topeka': 'KS',
 'Trenton': 'NJ'}

In [15]:
city_state_tuple = list(zip(capitals, postals))
city_state_tuple

[('Juneau', 'AK'),
 ('Montgomery', 'AL'),
 ('Little Rock', 'AR'),
 ('Phoenix', 'AZ'),
 ('Sacramento', 'CA'),
 ('Denver', 'CO'),
 ('Hartford', 'CT'),
 ('Dover', 'DE'),
 ('Tallahassee', 'FL'),
 ('Atlanta', 'GA'),
 ('Honolulu', 'HI'),
 ('Des Moines', 'IA'),
 ('Boise', 'ID'),
 ('Springfield', 'IL'),
 ('Indianapolis', 'IN'),
 ('Topeka', 'KS'),
 ('Frankfort', 'KY'),
 ('Baton Rouge', 'LA'),
 ('Boston', 'MA'),
 ('Annapolis', 'MD'),
 ('Augusta', 'ME'),
 ('Lansing', 'MI'),
 ('Saint Paul', 'MN'),
 ('Jefferson City', 'MO'),
 ('Jackson', 'MS'),
 ('Helena', 'MT'),
 ('Raleigh', 'NC'),
 ('Bismarck', 'ND'),
 ('Lincoln', 'NE'),
 ('Concord', 'NH'),
 ('Trenton', 'NJ'),
 ('Santa Fe', 'NM'),
 ('Carson City', 'NV'),
 ('Albany', 'NY'),
 ('Columbus', 'OH'),
 ('Oklahoma City', 'OK'),
 ('Salem', 'OR'),
 ('Harrisburg', 'PA'),
 ('Providence', 'RI'),
 ('Columbia', 'SC'),
 ('Pierre', 'SD'),
 ('Nashville', 'TN'),
 ('Austin', 'TX'),
 ('Salt Lake City', 'UT'),
 ('Richmond', 'VA'),
 ('Montpelier', 'VT'),
 ('Olympia', 'W

In [16]:
city_state_dictionary = dict((tuple_,[]) for tuple_ in city_state_tuple)
city_state_dictionary

{('Albany', 'NY'): [],
 ('Annapolis', 'MD'): [],
 ('Atlanta', 'GA'): [],
 ('Augusta', 'ME'): [],
 ('Austin', 'TX'): [],
 ('Baton Rouge', 'LA'): [],
 ('Bismarck', 'ND'): [],
 ('Boise', 'ID'): [],
 ('Boston', 'MA'): [],
 ('Carson City', 'NV'): [],
 ('Charleston', 'WV'): [],
 ('Cheyenne', 'WY'): [],
 ('Columbia', 'SC'): [],
 ('Columbus', 'OH'): [],
 ('Concord', 'NH'): [],
 ('Denver', 'CO'): [],
 ('Des Moines', 'IA'): [],
 ('Dover', 'DE'): [],
 ('Frankfort', 'KY'): [],
 ('Harrisburg', 'PA'): [],
 ('Hartford', 'CT'): [],
 ('Helena', 'MT'): [],
 ('Honolulu', 'HI'): [],
 ('Indianapolis', 'IN'): [],
 ('Jackson', 'MS'): [],
 ('Jefferson City', 'MO'): [],
 ('Juneau', 'AK'): [],
 ('Lansing', 'MI'): [],
 ('Lincoln', 'NE'): [],
 ('Little Rock', 'AR'): [],
 ('Madison', 'WI'): [],
 ('Montgomery', 'AL'): [],
 ('Montpelier', 'VT'): [],
 ('Nashville', 'TN'): [],
 ('Oklahoma City', 'OK'): [],
 ('Olympia', 'WA'): [],
 ('Phoenix', 'AZ'): [],
 ('Pierre', 'SD'): [],
 ('Providence', 'RI'): [],
 ('Raleigh', 'N

In [17]:
# SUMMER
summer_array = []
index = 0
# Iterate through the state abbreviation series
for i in summer['city']:
  # print(i)
  # check if state_abbr is in dict
  if i in capitals_postals_dict:
    # print(summer['state_abbr'])
    # print(summer['state_abbr'][index])
    if capitals_postals_dict[i] == summer['state_abbr'].iloc[index]:
      zipped = [i, summer['state_abbr'].iloc[index], summer['maxtempC'].iloc[index], summer['FeelsLikeC'].iloc[index], 
                summer['precipMM'].iloc[index], summer['mintempC'].iloc[index],  summer['totalSnow_cm'].iloc[index], 
                summer['uvIndex'].iloc[index], summer['DewPointC'].iloc[index], summer['HeatIndexC'].iloc[index], 
                summer['WindChillC'].iloc[index], summer['WindGustKmph'].iloc[index], 
                summer['cloudcover'].iloc[index], summer['humidity'].iloc[index], summer['pressure'].iloc[index], 
                summer['visibility'].iloc[index], summer['windspeedKmph'].iloc[index], summer['year'].iloc[index],
                summer['month'].iloc[index], summer['day'].iloc[index]]
      summer_array.append(zipped)
  index += 1

In [18]:
idx = 0
city_state_dictionary = dict((tuple_,[]) for tuple_ in city_state_tuple)
for i in city_state_dictionary:
  for list_ in summer_array:
    if (i[0] == list_[0]) & (i[1] == list_[1]):
      city_state_dictionary[i].append(list_)
  idx += 1
#city_state_dictionary[('Trenton', 'NJ')]
# dict(list(city_state_dictionary.items())[0:1])

In [19]:
for tuple_ in city_state_dictionary:
  length = len(city_state_dictionary[tuple_])
  print(length)

915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915
915


In [20]:
cols = ['maxtempC', 'FeelsLikeC', 'precipMM', 'mintempC', 'totalSnow_cm', 'uvIndex', 'DewPointC', 'HeatIndexC', 'WindChillC', 'WindGustKmph', 'cloudcover', 'humidity', 'pressure', 'visibility', 'windspeedKmph']

In [21]:
# for tuple_ in city_state_dictionary:
#   length = len(city_state_dictionary[tuple_])
#   temp_list = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] #, 0, 0, 0]
#   for i in range(len(city_state_dictionary[tuple_])):
#     temp_list[0] += city_state_dictionary[tuple_][i][2]
#     temp_list[1] += city_state_dictionary[tuple_][i][3]
#     temp_list[2] += city_state_dictionary[tuple_][i][4]
#     temp_list[3] += city_state_dictionary[tuple_][i][5]
#     temp_list[4] += city_state_dictionary[tuple_][i][6]
#     temp_list[5] += city_state_dictionary[tuple_][i][7]
#     temp_list[6] += city_state_dictionary[tuple_][i][8]
#     temp_list[7] += city_state_dictionary[tuple_][i][9]
#     temp_list[8] += city_state_dictionary[tuple_][i][10]
#     temp_list[9] += city_state_dictionary[tuple_][i][11]
#     temp_list[10] += city_state_dictionary[tuple_][i][12]
#     temp_list[11] += city_state_dictionary[tuple_][i][13]
#     temp_list[12] += city_state_dictionary[tuple_][i][14]
#     temp_list[13] += city_state_dictionary[tuple_][i][15]
#     temp_list[14] += city_state_dictionary[tuple_][i][16]
#     # temp_list[15] += city_state_dictionary[tuple_][i][17]
#     # temp_list[16] += city_state_dictionary[tuple_][i][18]
#     # temp_list[17] += city_state_dictionary[tuple_][i][19]


#   average = [num / length for num in temp_list]

# print(temp_list)
# print(average)

In [22]:
average_dict_summer = dict((tuple_,0) for tuple_ in city_state_tuple)
for tuple_ in city_state_dictionary:
  length = len(city_state_dictionary[tuple_])
  temp_list = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] #, 0, 0, 0]
  idx = 0
  # print(city_state_dictionary[tuple_])
  for i in city_state_dictionary[tuple_]:
    while idx < len(city_state_dictionary[tuple_])-1:
      # print(idx)
      temp_list[0] += city_state_dictionary[tuple_][idx][2]
      temp_list[1] += city_state_dictionary[tuple_][idx][3]
      temp_list[2] += city_state_dictionary[tuple_][idx][4]
      temp_list[3] += city_state_dictionary[tuple_][idx][5]
      temp_list[4] += city_state_dictionary[tuple_][idx][6]
      temp_list[5] += city_state_dictionary[tuple_][idx][7]
      temp_list[6] += city_state_dictionary[tuple_][idx][8]
      temp_list[7] += city_state_dictionary[tuple_][idx][9]
      temp_list[8] += city_state_dictionary[tuple_][idx][10]
      temp_list[9] += city_state_dictionary[tuple_][idx][11]
      temp_list[10] += city_state_dictionary[tuple_][idx][12]
      temp_list[11] += city_state_dictionary[tuple_][idx][13]
      temp_list[12] += city_state_dictionary[tuple_][idx][14]
      temp_list[13] += city_state_dictionary[tuple_][idx][15]
      temp_list[14] += city_state_dictionary[tuple_][idx][16]
      # temp_list[15] += city_state_dictionary[tuple_][i][17]
      # temp_list[16] += city_state_dictionary[tuple_][i][18]
      # temp_list[17] += city_state_dictionary[tuple_][i][19]
      idx += 1
  
  average = [num / length for num in temp_list]
  dictionary_value = dict(zip(cols, average))
  if (tuple_[0] == i[0]) & (tuple_[1] == i[1]):
      average_dict_summer[tuple_] = dictionary_value
# print(city_state_dictionary)
# print(idx)
# print(len(city_state_dictionary[tuple_]))

In [23]:
average_dict_summer

{('Albany', 'NY'): {'DewPointC': 11.410928961748635,
  'FeelsLikeC': 15.997814207650274,
  'HeatIndexC': 16.707103825136613,
  'WindChillC': 15.769398907103826,
  'WindGustKmph': 12.885245901639344,
  'cloudcover': 42.334426229508196,
  'humidity': 76.21420765027322,
  'maxtempC': 21.82950819672131,
  'mintempC': 10.40655737704918,
  'precipMM': 2.7192349726775946,
  'pressure': 1014.6491803278689,
  'totalSnow_cm': 0.0073224043715847,
  'uvIndex': 4.500546448087432,
  'visibility': 8.58688524590164,
  'windspeedKmph': 8.347540983606557},
 ('Annapolis', 'MD'): {'DewPointC': 15.911475409836065,
  'FeelsLikeC': 22.214207650273224,
  'HeatIndexC': 22.63169398907104,
  'WindChillC': 21.00655737704918,
  'WindGustKmph': 22.140983606557377,
  'cloudcover': 34.84699453551912,
  'humidity': 72.81748633879782,
  'maxtempC': 24.86775956284153,
  'mintempC': 18.055737704918034,
  'precipMM': 2.7277595628415288,
  'pressure': 1014.9191256830601,
  'totalSnow_cm': 0.0,
  'uvIndex': 5.50601092896174

In [24]:
# dictionary_value = dict(zip(cols, average))
# dictionary_value

In [25]:
# idx = 0
# for i in average_dict:
#   for j in average_dict:
#     if (i[0] == j[0]) & (i[1] == j[1]):
#       city_state_dictionary[i].append(j)
#   idx += 1
# #city_state_dictionary[('Trenton', 'NJ')]
# dict(list(city_state_dictionary.items())[0:2])

In [26]:
# WINTER
winter_array = []
index = 0
# Iterate through the state abbreviation series
for i in winter['city']:
  # print(i)
  # check if state_abbr is in dict
  if i in capitals_postals_dict:
    # print(winter['state_abbr'])
    # print(winter['state_abbr'][index])
    if capitals_postals_dict[i] == winter['state_abbr'].iloc[index]:
      zipped = [i, winter['state_abbr'].iloc[index], winter['maxtempC'].iloc[index], winter['FeelsLikeC'].iloc[index], 
                winter['precipMM'].iloc[index], winter['mintempC'].iloc[index],  winter['totalSnow_cm'].iloc[index], 
                winter['uvIndex'].iloc[index], winter['DewPointC'].iloc[index], winter['HeatIndexC'].iloc[index], 
                winter['WindChillC'].iloc[index], winter['WindGustKmph'].iloc[index], 
                winter['cloudcover'].iloc[index], winter['humidity'].iloc[index], winter['pressure'].iloc[index], 
                winter['visibility'].iloc[index], winter['windspeedKmph'].iloc[index], winter['year'].iloc[index],
                winter['month'].iloc[index], winter['day'].iloc[index]]
      winter_array.append(zipped)
  index += 1

In [27]:
idx = 0
city_state_dictionary = dict((tuple_,[]) for tuple_ in city_state_tuple)
for i in city_state_dictionary:
  for list_ in winter_array:
    if (i[0] == list_[0]) & (i[1] == list_[1]):
      city_state_dictionary[i].append(list_)
  idx += 1
#city_state_dictionary[('Trenton', 'NJ')]
# dict(list(city_state_dictionary.items())[0:1])

In [28]:
average_dict_winter = dict((tuple_,0) for tuple_ in city_state_tuple)
for tuple_ in city_state_dictionary:
  length = len(city_state_dictionary[tuple_])
  temp_list = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] #, 0, 0, 0]
  idx = 0
  # print(city_state_dictionary[tuple_])
  for i in city_state_dictionary[tuple_]:
    while idx < len(city_state_dictionary[tuple_])-1:
      # print(idx)
      temp_list[0] += city_state_dictionary[tuple_][idx][2]
      temp_list[1] += city_state_dictionary[tuple_][idx][3]
      temp_list[2] += city_state_dictionary[tuple_][idx][4]
      temp_list[3] += city_state_dictionary[tuple_][idx][5]
      temp_list[4] += city_state_dictionary[tuple_][idx][6]
      temp_list[5] += city_state_dictionary[tuple_][idx][7]
      temp_list[6] += city_state_dictionary[tuple_][idx][8]
      temp_list[7] += city_state_dictionary[tuple_][idx][9]
      temp_list[8] += city_state_dictionary[tuple_][idx][10]
      temp_list[9] += city_state_dictionary[tuple_][idx][11]
      temp_list[10] += city_state_dictionary[tuple_][idx][12]
      temp_list[11] += city_state_dictionary[tuple_][idx][13]
      temp_list[12] += city_state_dictionary[tuple_][idx][14]
      temp_list[13] += city_state_dictionary[tuple_][idx][15]
      temp_list[14] += city_state_dictionary[tuple_][idx][16]
      # temp_list[15] += city_state_dictionary[tuple_][i][17]
      # temp_list[16] += city_state_dictionary[tuple_][i][18]
      # temp_list[17] += city_state_dictionary[tuple_][i][19]
      idx += 1
  
  average = [num / length for num in temp_list]
  dictionary_value = dict(zip(cols, average))
  if (tuple_[0] == i[0]) & (tuple_[1] == i[1]):
      average_dict_winter[tuple_] = dictionary_value
# print(city_state_dictionary)
# print(idx)
# print(len(city_state_dictionary[tuple_]))

In [29]:
average_dict_winter

{('Albany', 'NY'): {'DewPointC': -2.6875,
  'FeelsLikeC': -2.337719298245614,
  'HeatIndexC': 0.6381578947368421,
  'WindChillC': -2.337719298245614,
  'WindGustKmph': 16.43530701754386,
  'cloudcover': 57.776315789473685,
  'humidity': 80.23574561403508,
  'maxtempC': 4.480263157894737,
  'mintempC': -3.2927631578947367,
  'precipMM': 1.9554824561403523,
  'pressure': 1017.2214912280701,
  'totalSnow_cm': 0.3410087719298247,
  'uvIndex': 1.8421052631578947,
  'visibility': 8.149122807017545,
  'windspeedKmph': 10.450657894736842},
 ('Annapolis', 'MD'): {'DewPointC': 2.107456140350877,
  'FeelsLikeC': 4.9769736842105265,
  'HeatIndexC': 7.5394736842105265,
  'WindChillC': 4.93640350877193,
  'WindGustKmph': 26.13486842105263,
  'cloudcover': 41.99122807017544,
  'humidity': 70.16228070175438,
  'maxtempC': 10.517543859649123,
  'mintempC': 4.421052631578948,
  'precipMM': 2.008552631578948,
  'pressure': 1018.6359649122807,
  'totalSnow_cm': 0.06282894736842105,
  'uvIndex': 2.83333333

In [30]:
def mergeDict(dict1, dict2):
   ''' Merge dictionaries and keep values of common keys in list'''
   dict3 = {**dict1, **dict2}
   for key, value in dict3.items():
       if key in dict1 and key in dict2:
               dict3[key] = [value , dict1[key]]
   return dict3
# Merge dictionaries and add values of common keys in a list
average_dict_final = mergeDict(average_dict_summer, average_dict_winter)
average_dict_final

{('Albany',
  'NY'): [{'DewPointC': -2.6875,
   'FeelsLikeC': -2.337719298245614,
   'HeatIndexC': 0.6381578947368421,
   'WindChillC': -2.337719298245614,
   'WindGustKmph': 16.43530701754386,
   'cloudcover': 57.776315789473685,
   'humidity': 80.23574561403508,
   'maxtempC': 4.480263157894737,
   'mintempC': -3.2927631578947367,
   'precipMM': 1.9554824561403523,
   'pressure': 1017.2214912280701,
   'totalSnow_cm': 0.3410087719298247,
   'uvIndex': 1.8421052631578947,
   'visibility': 8.149122807017545,
   'windspeedKmph': 10.450657894736842}, {'DewPointC': 11.410928961748635,
   'FeelsLikeC': 15.997814207650274,
   'HeatIndexC': 16.707103825136613,
   'WindChillC': 15.769398907103826,
   'WindGustKmph': 12.885245901639344,
   'cloudcover': 42.334426229508196,
   'humidity': 76.21420765027322,
   'maxtempC': 21.82950819672131,
   'mintempC': 10.40655737704918,
   'precipMM': 2.7192349726775946,
   'pressure': 1014.6491803278689,
   'totalSnow_cm': 0.0073224043715847,
   'uvIndex':

In [31]:
temp = average_dict_final
temp

{('Albany',
  'NY'): [{'DewPointC': -2.6875,
   'FeelsLikeC': -2.337719298245614,
   'HeatIndexC': 0.6381578947368421,
   'WindChillC': -2.337719298245614,
   'WindGustKmph': 16.43530701754386,
   'cloudcover': 57.776315789473685,
   'humidity': 80.23574561403508,
   'maxtempC': 4.480263157894737,
   'mintempC': -3.2927631578947367,
   'precipMM': 1.9554824561403523,
   'pressure': 1017.2214912280701,
   'totalSnow_cm': 0.3410087719298247,
   'uvIndex': 1.8421052631578947,
   'visibility': 8.149122807017545,
   'windspeedKmph': 10.450657894736842}, {'DewPointC': 11.410928961748635,
   'FeelsLikeC': 15.997814207650274,
   'HeatIndexC': 16.707103825136613,
   'WindChillC': 15.769398907103826,
   'WindGustKmph': 12.885245901639344,
   'cloudcover': 42.334426229508196,
   'humidity': 76.21420765027322,
   'maxtempC': 21.82950819672131,
   'mintempC': 10.40655737704918,
   'precipMM': 2.7192349726775946,
   'pressure': 1014.6491803278689,
   'totalSnow_cm': 0.0073224043715847,
   'uvIndex':

In [32]:
for i in temp:
  string = str(i)
  i = string
temp

{('Albany',
  'NY'): [{'DewPointC': -2.6875,
   'FeelsLikeC': -2.337719298245614,
   'HeatIndexC': 0.6381578947368421,
   'WindChillC': -2.337719298245614,
   'WindGustKmph': 16.43530701754386,
   'cloudcover': 57.776315789473685,
   'humidity': 80.23574561403508,
   'maxtempC': 4.480263157894737,
   'mintempC': -3.2927631578947367,
   'precipMM': 1.9554824561403523,
   'pressure': 1017.2214912280701,
   'totalSnow_cm': 0.3410087719298247,
   'uvIndex': 1.8421052631578947,
   'visibility': 8.149122807017545,
   'windspeedKmph': 10.450657894736842}, {'DewPointC': 11.410928961748635,
   'FeelsLikeC': 15.997814207650274,
   'HeatIndexC': 16.707103825136613,
   'WindChillC': 15.769398907103826,
   'WindGustKmph': 12.885245901639344,
   'cloudcover': 42.334426229508196,
   'humidity': 76.21420765027322,
   'maxtempC': 21.82950819672131,
   'mintempC': 10.40655737704918,
   'precipMM': 2.7192349726775946,
   'pressure': 1014.6491803278689,
   'totalSnow_cm': 0.0073224043715847,
   'uvIndex':

In [41]:
# temp2 = "  ".join([i[0] for i in temp])
# temp3 = "  ".join([i[1] for i in temp])
# for i in range(0, len(temp)):
#   var = states[i] + ", " + postals[i]
#   # print(var)

new = [capitals[i] + ", " + postals[i] for i in range(0, len(temp))]
new
# for i in new:
#   average_dict_final[i]
new = sorted(new)

# new_dict = {value:key for (key,value) in old_dict.items()}
zipped = dict(zip(new, list(temp.values())))

for i in temp:
  zipped[i] = temp[i]
# zipped
for i in zipped:
  # print(type(i))
  if type(i) == 'tuple':
    zipped.remove(i)
# zipped

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tupl

In [49]:
def remap_keys(mapping):
  return [{'key':k, 'value': v} for k, v in mapping.items()]

final = json.dumps(remap_keys(temp))
final

'[{"key": ["Juneau", "AK"], "value": [{"maxtempC": -1.3793859649122806, "FeelsLikeC": -6.8355263157894735, "precipMM": 8.517434210526318, "mintempC": -5.640350877192983, "totalSnow_cm": 3.773684210526315, "uvIndex": 1.3475877192982457, "DewPointC": -5.12609649122807, "HeatIndexC": -3.3717105263157894, "WindChillC": -6.8355263157894735, "WindGustKmph": 15.50109649122807, "cloudcover": 69.24890350877193, "humidity": 87.8344298245614, "pressure": 1010.9309210526316, "visibility": 6.830043859649122, "windspeedKmph": 9.12390350877193}, {"maxtempC": 9.237158469945355, "FeelsLikeC": 5.456830601092896, "precipMM": 6.230273224043704, "mintempC": 3.93224043715847, "totalSnow_cm": 0.5083060109289619, "uvIndex": 2.273224043715847, "DewPointC": 4.510382513661202, "HeatIndexC": 6.615300546448087, "WindChillC": 5.456830601092896, "WindGustKmph": 10.302732240437159, "cloudcover": 69.68196721311476, "humidity": 87.46120218579235, "pressure": 1013.8775956284153, "visibility": 7.0415300546448085, "windsp

In [ ]:
# loaded_final = json.loads(final)
# loaded_final

In [ ]:
final = json.dumps(average_dict_final)
loaded_final = json.loads(final)
loaded_final

## Base Weather Models

In [ ]:
pip install scikit-learn

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
data.shape

In [ ]:
data.plot(x='mintempC', y='maxtempC', style='o')  
plt.title('MinTemp vs MaxTemp')  
plt.xlabel('MinTemp')  
plt.ylabel('MaxTemp')  
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
plt.tight_layout()
sns.distplot(data['maxtempC'])

In [ ]:
# Get baseline for temperature
baseline_temp = sum(data['maxtempC']) / data.shape[0]
baseline_temp

In [ ]:
# # Separate data into target and features
# X = data.drop(['location','maxtempC','date_time'], axis=1)
# y = data['maxtempC']

In [ ]:
# # train test split data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [ ]:
# # Fit Linear Regression Model to Data
# LinReg = LinearRegression()
# LinReg.fit(X_train, y_train)

In [ ]:
# y_pred = LinReg.predict(X_test)

In [ ]:
# df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
# df

In [ ]:
# print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
# print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
# print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))